## Import Required Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from phantominator import shepp_logan

## Original vs Modified Shepp-Logan

The original Shepp-Logan phantom and the modified version with better contrast:

In [ ]:
# The original Shepp-Logan
ph = shepp_logan(128, modified=False)
plt.subplot(1, 2, 1)
plt.title('Shepp-Logan')
plt.imshow(ph, cmap='gray')

# Modified Shepp-Logan for better contrast
ph = shepp_logan(128, modified=True)
plt.subplot(1, 2, 2)
plt.title('Modified Shepp-Logan')
plt.imshow(ph, cmap='gray')
plt.show()

## Non-Square Phantom

Generate a phantom with different width and height dimensions:

In [ ]:
# Generate phantoms with different sizes
ph = shepp_logan((128, 256))
plt.title('Strange sizes')
plt.imshow(ph, cmap='gray')
plt.show()

## 3D Phantom

Create a 3D phantom and display all slices:

In [ ]:
# Get a 3D phantom
ph = shepp_logan((128, 128, 20), zlims=(-.5, .5))

# Fancy dancing to nicely show all slices on same plot
nx = int(np.ceil(np.sqrt(ph.shape[-1])))
fig = plt.figure(figsize=(nx, nx))
for ii in range(ph.shape[-1]):
    ax = fig.add_subplot(nx, nx, ii+1)
    plt.imshow(ph[..., ii], cmap='gray')
    ax.get_xaxis().set_ticks([])
    ax.get_yaxis().set_ticks([])
    ax.set_aspect('equal')
fig.subplots_adjust(wspace=0, hspace=0)
plt.show()

## Generate Dataset

Generate multiple Shepp-Logan phantom samples and save them as PNG files:

In [ ]:
import os
import json

# Configuration
phantom_type = 'modified'  # 'original' or 'modified'
nsamples = 10  # Number of samples to generate
image_size = 128  # Size of the phantom
output_folder = 'generated_data'  # Folder to save PNG files
seed = 42  # Random seed for reproducibility

# Prompt style for captions
prompt_style = 'detailed'  # 'simple', 'detailed', or 'medical'
# - simple: "brain scan with large ventricles"
# - detailed: "A medical brain scan showing a large brain with enlarged ventricles"
# - medical: "CT scan of human brain, large brain size, enlarged lateral ventricles"

# Anatomical variation parameters - simulate different brain anatomies
vary_brain_size = True  # Vary overall brain size
brain_size_range = (0.85, 1.15)  # Scale factor range for brain size

vary_ventricles = True  # Vary ventricle (fluid cavity) size
ventricle_size_range = (0.7, 2.0)  # Larger range for more variation

vary_white_matter = True  # Vary white matter structure
white_matter_range = (0.8, 1.3)  # White matter variation

vary_tumors = True  # Vary tumor/lesion sizes (small ellipses)
tumor_range = (0.5, 1.8)  # Tumor size variation

vary_shape_elongation = True  # Vary ellipse elongation (aspect ratio)
elongation_std = 0.1  # Standard deviation for aspect ratio changes

vary_rotation = True  # Vary rotation angles of ellipses
rotation_std = 10  # Degrees of rotation variation

vary_position = True  # Shift positions of internal structures
position_std = 0.04  # Position shift standard deviation

vary_intensity = True  # Vary tissue intensities slightly
intensity_std = 0.08  # Intensity variation

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Set random seed for reproducibility
np.random.seed(seed)

# Get base ellipse parameters
modified = phantom_type == 'modified'
base_phantom = shepp_logan(image_size, modified=modified, ret_E=True)
_, E_base = base_phantom

print("Generating different 'brain' anatomies with multiple variations...\n")
print(f"Caption style: {prompt_style}\n")

# Store dataset-level metadata
dataset_metadata = {
    'dataset_name': 'synthetic_brain_phantoms',
    'phantom_type': phantom_type,
    'image_size': image_size,
    'num_samples': nsamples,
    'seed': seed,
    'prompt_style': prompt_style,
    'variation_parameters': {
        'brain_size': brain_size_range if vary_brain_size else None,
        'ventricles': ventricle_size_range if vary_ventricles else None,
        'white_matter': white_matter_range if vary_white_matter else None,
        'tumors': tumor_range if vary_tumors else None,
    }
}

# Generate and save samples with anatomical variations
for i in range(nsamples):
    # Copy base parameters
    E = E_base.copy()
    
    # Dictionary to store metadata about this sample
    metadata = {
        'sample_id': i,
        'phantom_type': phantom_type,
        'image_size': image_size,
        'seed': seed,
        'variations': {}
    }
    
    # Keep ellipse 0 (outermost/background) unchanged to maintain consistent background
    # Only vary internal structures (ellipses 1-9)
    
    # 1. Vary overall brain size (ellipses 1-3: inner skull, CSF, brain tissue)
    if vary_brain_size:
        brain_scale = np.random.uniform(*brain_size_range)
        metadata['variations']['brain_size_scale'] = float(brain_scale)
        for idx in [1, 2, 3]:
            E[idx, 1] *= brain_scale  # major axis
            E[idx, 2] *= brain_scale  # minor axis
    
    # 2. Vary ventricle sizes (ellipses 4, 5 are lateral ventricles)
    ventricle_scales = []
    if vary_ventricles:
        # Each ventricle can vary independently
        for idx in [4, 5]:
            ventricle_scale = np.random.uniform(*ventricle_size_range)
            ventricle_scales.append(float(ventricle_scale))
            E[idx, 1] *= ventricle_scale
            E[idx, 2] *= ventricle_scale
        metadata['variations']['ventricle_scales'] = ventricle_scales
    
    # 3. Vary white matter (ellipse 6)
    if vary_white_matter:
        wm_scale = np.random.uniform(*white_matter_range)
        metadata['variations']['white_matter_scale'] = float(wm_scale)
        E[6, 1] *= wm_scale
        E[6, 2] *= wm_scale
    
    # 4. Vary tumor/small structure sizes (ellipses 7-9)
    tumor_scales = []
    if vary_tumors and E.shape[0] > 7:
        for idx in range(7, min(10, E.shape[0])):
            tumor_scale = np.random.uniform(*tumor_range)
            tumor_scales.append(float(tumor_scale))
            E[idx, 1] *= tumor_scale
            E[idx, 2] *= tumor_scale
        metadata['variations']['tumor_scales'] = tumor_scales
    
    # 5. Vary shape elongation (aspect ratio) for internal structures
    if vary_shape_elongation:
        metadata['variations']['shape_elongation'] = True
        for idx in range(1, E.shape[0]):
            elongation_factor = np.random.normal(1.0, elongation_std)
            E[idx, 2] *= elongation_factor
    
    # 6. Vary rotation angles
    if vary_rotation:
        metadata['variations']['rotation_applied'] = True
        for idx in range(1, E.shape[0]):
            angle_delta = np.random.normal(0, np.deg2rad(rotation_std))
            E[idx, 5] += angle_delta
    
    # 7. Add position shifts for internal structures
    if vary_position:
        metadata['variations']['position_shifts'] = True
        E[1:, 3] += np.random.normal(0, position_std, E.shape[0]-1)
        E[1:, 4] += np.random.normal(0, position_std, E.shape[0]-1)
    
    # 8. Vary tissue intensities
    if vary_intensity:
        metadata['variations']['intensity_variation'] = True
        for idx in range(1, E.shape[0]):
            intensity_factor = np.random.normal(1.0, intensity_std)
            E[idx, 0] *= intensity_factor
    
    # Generate descriptive features for caption
    features = []
    
    # Brain size descriptor
    brain_size_desc = None
    if vary_brain_size:
        if brain_scale > 1.08:
            brain_size_desc = "large brain"
        elif brain_scale < 0.92:
            brain_size_desc = "small brain"
        else:
            brain_size_desc = "normal-sized brain"
        features.append(brain_size_desc)
    
    # Ventricle descriptor
    ventricle_desc = None
    if vary_ventricles:
        avg_ventricle = np.mean(ventricle_scales)
        if avg_ventricle > 1.3:
            ventricle_desc = "significantly enlarged ventricles"
        elif avg_ventricle > 1.1:
            ventricle_desc = "enlarged ventricles"
        elif avg_ventricle < 0.85:
            ventricle_desc = "reduced ventricles"
        else:
            ventricle_desc = "normal ventricles"
        features.append(ventricle_desc)
    
    # White matter descriptor
    wm_desc = None
    if vary_white_matter:
        if wm_scale > 1.15:
            wm_desc = "expanded white matter"
        elif wm_scale < 0.85:
            wm_desc = "reduced white matter"
        else:
            wm_desc = "normal white matter"
        features.append(wm_desc)
    
    # Lesion/tumor descriptor
    tumor_desc = None
    if vary_tumors and tumor_scales:
        avg_tumor = np.mean(tumor_scales)
        if avg_tumor > 1.4:
            tumor_desc = "prominent lesions"
        elif avg_tumor > 1.1:
            tumor_desc = "visible lesions"
        elif avg_tumor < 0.7:
            tumor_desc = "minimal lesions"
        else:
            tumor_desc = "small lesions"
        features.append(tumor_desc)
    
    # Create caption based on prompt style
    if prompt_style == 'simple':
        caption = "brain scan"
        if features:
            caption += " with " + ", ".join(features)
    
    elif prompt_style == 'medical':
        scan_type = "CT scan" if phantom_type == 'modified' else "medical scan"
        caption = f"{scan_type} of human brain"
        if features:
            caption += ", " + ", ".join(features)
    
    else:  # detailed (default)
        article = "An" if features and features[0][0] in 'aeiou' else "A"
        caption = f"{article} medical brain scan showing"
        if features:
            caption += " " + ", ".join(features)
        else:
            caption += " standard brain anatomy"
    
    metadata['caption'] = caption
    
    # Generate phantom with modified parameters
    ph = shepp_logan(image_size, modified=modified, E=E)
    
    # Normalize intensity to keep consistent range
    ph = np.clip(ph, 0, 1)
    
    # Save as PNG
    filename_base = f'brain_{i:04d}'
    output_path = os.path.join(output_folder, f'{filename_base}.png')
    plt.figure(figsize=(6, 6))
    plt.imshow(ph, cmap='gray', vmin=0, vmax=1)
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(output_path, bbox_inches='tight', dpi=150)
    plt.close()
    
    # Save caption as TXT (main file for diffusion training)
    txt_path = os.path.join(output_folder, f'{filename_base}.txt')
    with open(txt_path, 'w') as f:
        f.write(caption)
    
    # Save detailed metadata as JSON (optional, for reference)
    json_path = os.path.join(output_folder, f'{filename_base}_metadata.json')
    with open(json_path, 'w') as f:
        json.dump(metadata, f, indent=2)
    
    print(f'[{i+1}/{nsamples}] {filename_base}.png')
    print(f'  Caption: "{caption}"')

# Save dataset-level metadata
dataset_json_path = os.path.join(output_folder, 'dataset_metadata.json')
with open(dataset_json_path, 'w') as f:
    json.dump(dataset_metadata, f, indent=2)

print(f'\n✓ Generated {nsamples} brain samples in "{output_folder}/"')
print(f'\nFiles per sample:')
print(f'  • brain_XXXX.png - Image file')
print(f'  • brain_XXXX.txt - Caption/prompt (for training)')
print(f'  • brain_XXXX_metadata.json - Detailed metadata (optional)')
print(f'\n✓ Dataset metadata saved to dataset_metadata.json')
print(f'\nReady for LoRA training with frameworks like:')
print(f'  - Kohya SD-Scripts')
print(f'  - AUTOMATIC1111/sd-webui')
print(f'  - HuggingFace Diffusers')